In [3]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt

from qiskit import Aer
from tqdm.notebook import tqdm
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [95]:
class QuantumKernel:
    def __init__(self, n_qubits, reps, shots):
        self.n_qubits = n_qubits
        self.shots = shots
        self.reps = reps
        
        self.n_params = self.reps*self.n_qubits + 1
        self.params = np.random.uniform(-np.pi, np.pi, self.n_params)
        
        
    def encoder(self, circuit, storage, x):
        for i, _x in enumerate(x):
            circuit.rx(_x, storage[i])
        
        return circuit
        
        
    def ansatz(self, circuit, storage, params):
        for i, param in enumerate(params):
            circuit.ry(param, storage[i])
            
        for i in range(self.n_qubits-1):
            circuit.cx(storage[i], storage[i+1])
            
        return circuit
    
    
    def ansatz_bias(self, circuit, storage, param):
        circuit.ry(param, storage[-1])
        
        return circuit
    
    
    def evaluate(self, data):
        outputs = []
        for x in data:
            storage = qk.QuantumRegister(self.n_qubits)
            clas_reg = qk.ClassicalRegister(1)

            circuit = qk.QuantumCircuit(storage, clas_reg)
            circuit = self.encoder(circuit, storage, x)
            
            idx_start = 0
            idx_end = self.n_qubits
            
            for i in range(self.reps):
                circuit = self.ansatz(circuit, storage, self.params[idx_start:idx_end])
                idx_start = idx_end
                idx_end += self.n_qubits
                
            circuit = self.ansatz_bias(circuit, storage, self.params[-1])
            circuit.measure(storage[-1], clas_reg)
            
            job = qk.execute(circuit, backend=qk.Aer.get_backend(
                'qasm_simulator'), shots=self.shots)
            
            counts = job.result().get_counts(circuit)
            
            output = 0
            for bitstring, samples in counts.items():
                if bitstring == "1":
                    output += samples

            output = output / self.shots
            
            outputs.append(output)
        
        return np.array(outputs).reshape(-1,1)
    
    def gradient(self, data, y_pred, y):
        n_samples = data.shape[0]
        
        cost_deriv = (y_pred - y)
        partial_grad = np.zeros((self.n_params, data.shape[0]))
        
        for i in range(self.n_params):
            self.params[i] += np.pi/2
            shift_plus = self.evaluate(data)
            
            self.params[i] -= np.pi
            shift_minus = self.evaluate(data)
            
            self.params[i] += np.pi/2
            
            partial_grad[i] = 0.5*(shift_plus - shift_minus).flatten()
            
        
        gradients = 1/n_samples * partial_grad@cost_deriv 
        
        return gradients.flatten()
    
    def train(self, data, y, epochs=100, lr = 1):
        for i in tqdm(range(epochs)):
            y_pred = self.evaluate(data)
            gradient = self.gradient(data, y_pred, y)
            self.params = self.params - lr*gradient
            
            print(np.mean((y_pred - y)**2))
      

# Iris Data

## Data Preparation

In [96]:
iris = load_iris()

### Only first two classes

In [97]:
x = iris.data
y = iris.target

x = x[y != 2]
y = y[y != 2].reshape(-1,1)

np.random.seed(42)
x, _, y, _ = train_test_split(x, y, train_size=10)
print(x.shape, y.shape)

(10, 4) (10, 1)


### Normalize Input Data

In [89]:
x = scaler(x, a=0, b=np.pi)

## Train Model

In [100]:
np.random.seed(44)
model = QuantumKernel(n_qubits = 4, reps = 2, shots=10000)

In [102]:
model.train(x, y, epochs = 10, lr=1)

  0%|          | 0/10 [00:00<?, ?it/s]

0.20300338199999998
0.20438050200000002
0.20214160099999998
0.200733632


KeyboardInterrupt: 

In [31]:
y_pred = model.evaluate(x)

In [32]:
print(np.mean(np.round(y_pred).astype(int) == y))

0.99
